In [ ]:
#This Python script is designed to calculate allele frequencies at variant positions specified in a VCF file by inspecting a corresponding BAM file.
#The code does the following:

#It reads through the VCF file to identify variant positions (chromosome and position).
#It uses the BAM file to count the occurrences of each allele (A, C, G, T) at those positions.
#It calculates the allele frequencies at these positions and writes them to an output file.

import pysam
from cyvcf2 import VCF

bam_path = "/home/dingrongruo.yu/liz.9.11.19_GMVLE/results/mapping/HG003_NA24149_Ashkenazim_father.trim.sort.markdup.bam"
vcf_file_path = '/home/dingrongruo.yu/liz.9.11.19_GMVLE/results/variants/HG003_NA24149_Ashkenazim_father.trim.dv.vcf'
output_file_path = '/home/dingrongruo.yu/liz.9.11.19_GMVLE/features/output.txt'

def calculate_allele_frequency(bam_file, chromosome, position):
    # Fetch pileup for the specific position
    pileup_columns = bam_file.pileup(chromosome, position-1, position, stepper='nofilter')

    allele_counts = {}
    total_count = 0

    for pileup_column in pileup_columns:
        if pileup_column.reference_pos == position-1:  # Check if the position matches
            for pileup_read in pileup_column.pileups:
                if not pileup_read.is_del and not pileup_read.is_refskip:  # Ignore deletions and ref skips
                    base = pileup_read.alignment.query_sequence[pileup_read.query_position]
                    allele_counts[base] = allele_counts.get(base, 0) + 1
                    total_count += 1

    # Check for division by zero
    if total_count > 0:
        allele_frequencies = {allele: count / total_count for allele, count in allele_counts.items()}
    else:
        allele_frequencies = {}

    return allele_frequencies

def calculate_frequencies_for_variants(bam_path, vcf_file_path, output_file_path):
    bam_file = pysam.AlignmentFile(bam_path, "rb")
    vcf = VCF(vcf_file_path)

    with open(output_file_path, 'w') as output_file:
        for record in vcf:
            chrom = record.CHROM
            pos = record.POS
            freqs = calculate_allele_frequency(bam_file, chrom, pos)
            
            output_file.write(f"{chrom}\t{pos}\t{freqs}\n")

    bam_file.close()

# Run the frequency calculation
calculate_frequencies_for_variants(bam_path, vcf_file_path, output_file_path)
